### import libraries

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
import json
import string

### load dataset

In [3]:
df = pd.read_csv('../HomeworkThree/dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23462 entries, 0 to 23461
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                1100 non-null   float64
 1   course_name               1100 non-null   object 
 2   course_instructor_site    1090 non-null   object 
 3   course_site               1054 non-null   object 
 4   course_instructor         1070 non-null   object 
 5   course_cost               897 non-null    object 
 6   course_credential         819 non-null    object 
 7   course_level              725 non-null    object 
 8   course_duration           1041 non-null   object 
 9   course_language           1054 non-null   object 
 10  course_caption_languages  716 non-null    object 
 11  overview                  1060 non-null   object 
 12  syllabus                  848 non-null    object 
 13  subject                   1100 non-null   object 
dtypes: flo

In [ ]:
df = df.loc[~(df['Requirements'].isnull())]
df = df.dropna(subset = ['Requirements'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4339 entries, 0 to 6212
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   University    4339 non-null   object
 1   Department    4339 non-null   object
 2   Course title  4339 non-null   object
 3   Requirements  4339 non-null   object
 4   Course Group  64 non-null     object
dtypes: object(5)
memory usage: 203.4+ KB


### clean texts

In [ ]:
def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern,'',text)
    return text

In [ ]:
def tokenize_text(text):
  return text.lower().split()

In [ ]:
nltk.download('stopwords')
stopword = stopwords.words('english')

def remove_stopwords(tokenized_overview):
  cleaned_list = []
  for word in tokenized_overview:
    if word not in stopword:
      cleaned_list.append(word)
  return cleaned_list

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
ps = PorterStemmer()

def stemmer(tokenized_overview):
  stemmed_list = []
  for word in tokenized_overview:
    stemmed_list.append(ps.stem(word))
  return stemmed_list

In [ ]:
nltk.download('wordnet')
wn = WordNetLemmatizer()

def lemmatizer(tokenized_overview):
  lemmatized_list = []
  for word in tokenized_overview:
    lemmatized_list.append(wn.lemmatize(word))
  return lemmatized_list

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
def l2s(tokenize_text):
  text = " " 
  return (text.join(tokenize_text))

In [ ]:
df['Requirements'] = df['Requirements'].apply(remove_special_characters)
df['tokenized_Requirements'] = df['Requirements'].apply(tokenize_text)
df['tokenized_Requirements'] = df['tokenized_Requirements'].apply(remove_stopwords)
df['tokenized_Requirements'] = df['tokenized_Requirements'].apply(stemmer)
df['tokenized_Requirements'] = df['tokenized_Requirements'].apply(lemmatizer)
df['cleaned_Requirements'] = df['tokenized_Requirements'].apply(l2s)
df.head(10)

,University,Department,Course title,Requirements,Course Group,tokenized_Requirements,cleaned_Requirements
0,The Australian National University,ANU College of Engineering and Computer Science,Applied Cybernetics Professional Experience,To enrol in this course you must be enrolled i...,"CECS Experimental, Interdisciplinary","[enrol, cours, must, enrol, master, appli, cyb...",enrol cours must enrol master appli cybernet
2,University of California Berkeley,Computer Science (COMPSCI),Foundations of Data Science,This course may be taken on its own but studen...,NaN,"[cours, may, taken, student, encourag, take, c...",cours may taken student encourag take concurr ...
3,University of California Berkeley,Computer Science (COMPSCI),Matlab for Programmers,Programming experience equivalent to that gain...,NaN,"[program, experi, equival, gain, compsci, 10, ...",program experi equival gain compsci 10 familia...
4,University of California Berkeley,Computer Science (COMPSCI),C for Programmers,Programming experience with pointers or addres...,NaN,"[program, experi, pointer, address, assembl, l...",program experi pointer address assembl languag...
5,University of California Berkeley,Computer Science (COMPSCI),Scheme and Functional Programming for Programmers,Programming experience similar to that gained ...,NaN,"[program, experi, similar, gain, compsci, 10, ...",program experi similar gain compsci 10 engin 7
6,University of California Berkeley,Computer Science (COMPSCI),Productive Use of the UNIX Environment,Programming experience similar to that gained ...,NaN,"[program, experi, similar, gain, compsci, 61a,...",program experi similar gain compsci 61a engin ...
7,University of California Berkeley,Computer Science (COMPSCI),C++ for Programmers,Programming experience equivalent to that gain...,NaN,"[program, experi, equival, gain, compsci, 61a,...",program experi equival gain compsci 61a engin 7
8,University of California Berkeley,Computer Science (COMPSCI),JAVA for Programmers,COMPSCI 9C COMPSCI 9F or COMPSCI 61A plus expe...,NaN,"[compsci, 9c, compsci, 9f, compsci, 61a, plu, ...",compsci 9c compsci 9f compsci 61a plu experi o...
9,University of California Berkeley,Computer Science (COMPSCI),Python for Programmers,Programming experience equivalent to that gain...,NaN,"[program, experi, equival, gain, compsci, 10]",program experi equival gain compsci 10
12,University of California Berkeley,Computer Science (COMPSCI),CS Scholars Seminar: The Educational Climate i...,Prerequisite satisfied Concurrently Participat...,NaN,"[prerequisit, satisfi, concurr, particip, c, s...",prerequisit satisfi concurr particip c scholar...
